In [51]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd


#Load the dataset

data = gutenberg.raw('shakespeare-hamlet.txt')

## Save to a file
with open('hamlet2.txt','w') as file:
    file.write(data)


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sukhdeepnarulasingh/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [52]:
## Data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


# Load the dataset

with open('hamlet2.txt', 'r') as file:
    text = file.read().lower()

# Tokenize the text-creating indexs for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1




In [53]:
## Crteate input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

        

In [54]:
## Pad sequences
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [55]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [56]:
## Create predcitors and label

import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [57]:
## Train our LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,GRU


## Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words, activation='softmax'))


## Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build((None, max_sequence_length))
model.summary()

/Users/sukhdeepnarulasingh/AI-ML/GenAI-Udemy/LSTM-RNN/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_10 (GRU)                    │ (None, 14, 150)        │       113,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_11 (GRU)                    │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,157,418 (4.42 MB)

 Trainable params: 1,157,418 (4.42 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)


In [59]:
## Train the model
history = model.fit(x_train, y_train, epochs=50, verbose=1, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.0265 - loss: 7.1969 - val_accuracy: 0.0391 - val_loss: 6.7862
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.0412 - loss: 6.4770 - val_accuracy: 0.0488 - val_loss: 6.7910
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0537 - loss: 6.1954 - val_accuracy: 0.0560 - val_loss: 6.7479
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.0691 - loss: 5.9403 - val_accuracy: 0.0661 - val_loss: 6.7734
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.0863 - loss: 5.6341 - val_accuracy: 0.0686 - val_loss: 6.8105
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.1002 - loss: 5.3365 - val_accuracy: 0.0668 - val_loss: 6.8902
Epoch 6: early stopping


In [60]:
model.evaluate(x_test, y_test)

161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0667 - loss: 6.8713


[6.890235900878906, 0.06683505326509476]

In [61]:
def predict_next_word(model, tokenizer, text, max_sequence_length):
    # Convert the text to a sequence of integers
    token_list = tokenizer.texts_to_sequences([text])[0]

    if len(token_list) > max_sequence_length - 1:
        token_list = token_list[-(max_sequence_length - 1):]
    
    # Pad the sequence
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    
    # Predict the next word
    predicted = model.predict(token_list, verbose=0)
    
    # Get the index of the predicted word
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [62]:
input_text = "Mar. Horatio saies, 'tis but our"
print(f"Input text: {input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Predicted next word: {next_word}")

Input text: Mar. Horatio saies, 'tis but our
Predicted next word: king


In [63]:
## Save the model
model.save('next_word_gru.keras')

## Save the tokenizer
import pickle
with open('tokenizer_gru.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)